In [ ]:
!pip install openai
import requests
import json

In [ ]:
import openai
API_KEY = "XXX"
API_ENDPOINT = "https://api.openai.com/v1/chat/completions"
openai.api_key = API_KEY

In [ ]:
with open('/content/drive/MyDrive/MacawProject/optional_dims.txt', 'r') as optional_dims:
  optional_dims_content = optional_dims.read()

In [ ]:
import openai
def gpt3(prompt):
    response = openai.Completion.create(
        model="text-davinci-003",
        prompt=prompt,
        temperature=0.7,
        max_tokens=1000,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0
    )
    if response:
        if response.choices and response.choices[0]:
            res = response.choices[0].text.strip()
            return res
    return None


In [ ]:
import json

def read_situation_from_json(file_path, is_train):
  situation_data = []
  with open(file_path, 'r') as file:
    for line in file:
      data = json.loads(line)
      question = data.get('question')
      possible_ans = data.get('mcoptions')
      d = {}
      d['question'] = question
      d['answers'] = possible_ans
      if is_train:
        chosen_dims = data.get('chosen_dimensions')
        elaborations = data.get('elaboration')
        d['chosen_dimensions'] = chosen_dims
        d['elaboration'] = elaborations
      situation_data.append(d)
  return situation_data

situations = read_situation_from_json("/content/drive/MyDrive/MacawProject/in_context_few_shots.jsonl", True)


In [ ]:



def append_dims_elaborations_to_few_shot_prompt(filename, situation, dims, elaborations):
  with open(filename, 'a') as prompt_file:
    prompt_file.write('Inputs:\n')
    prompt_file.write('SITUATION: ' + situation['question'] + '\n')
    prompt_file.write('ANSWERS: ' + situation['answers'] + '\n')
    prompt_file.write('Outputs:\n')
    prompt_file.write('CHOSEN_DIMENSIONS:\n' + dims)
    prompt_file.write('ELABORATIONS:\n' + elaborations + '\n')


filename = '/content/drive/MyDrive/MacawProject/dimensions_prompt.txt'
for situation in situations:
  dims = ""
  elaborations = ""
  for d in situation['chosen_dimensions']:
    dims += d + '\n'
  elaborations += situation['elaboration'].replace(" [", "\n[")

  append_dims_elaborations_to_few_shot_prompt(filename, situation, dims, elaborations)






In [ ]:
import pdb
situations = read_situation_from_json("/content/drive/MyDrive/MacawProject/test_samples.jsonl", False)

with open ('/content/drive/MyDrive/MacawProject/dimensions_prompt.txt', 'r') as prompt_few_shot:
  prompt_few_shot_content = prompt_few_shot.read()


result = []
for situation in situations:
  prompt = prompt_few_shot_content + '\n' + "SITUATION: " + situation['question']
  res = gpt3(prompt)
  res = res.split("CHOSEN_DIMENSIONS:")[1]
  chosen_dims, elaborations = res.split("ELABORATIONS:")
  chosen_dims_list = [item for item in chosen_dims.split('\n') if item]
  elaborations = elaborations.replace('\n', '')
  situation['chosen_dimensions'] = chosen_dims_list
  situation['elaboration'] = elaborations
  result.append(situation)

print(result)





In [ ]:
def write_result_to_jsonl(data, filename):
    with open(filename, 'w') as f:
        for item in data:
            f.write(json.dumps(item))
            f.write('\n')

filename = '/content/drive/MyDrive/MacawProject/gpt3_output.jsonl'
write_result_to_jsonl(result, filename)

In [ ]:
# import pandas as pd
# import seaborn as sns
# import matplotlib.pyplot as plt
# from sklearn.metrics import confusion_matrix
# import numpy as np
# from sklearn.metrics import classification_report


# dimensions_csv = pd.read_csv('/content/drive/MyDrive/MacawProject/dimensionsCsvToHashMap.csv')
# r = []
# for index, row in dimensions_csv.iterrows():
#   d = {'dimension' : row[0], 'description' : row[1], 'Relation of' : row[3]}
#   r.append(d)

# r